In [43]:
import sys
sys.path.insert(0, '/home/johnr_000/jupyter_blog/jupyter_blog/scripts')
import eia_model as em
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.plotly as py
import cufflinks as cf
from scipy import stats

In [3]:
class GetSeriesRange(em.GetSeries):
    '''
    Performs a call to the EIA API based on date range and captures the response
    valid kwargs:
        :param api_key: an valid API key provided by EIA
        :param series: a valid EIA series ID
        :param start: a start date in '%Y-%m-%d %H:%M:%S' fromat
        :param end: a end date in '%Y-%m-%d %H:%M:%S' fromat
        :freq: a valid frequency ('A' : annual, 'M': monthly, 'W': weekly, 
            'D': daily, 'H': hourly)
    '''
    
    def create_parms(self):
        '''
        Convert kwargs into a list to pass into api call
        '''
        try:
            kwargs_list = [['api_key', self.kwargs['api_key']]]
            kwargs_list.append(['series_id', self.kwargs['series_id']])
            kwargs_list.append(['start', self.format_date(self.kwargs['freq'], self.kwargs['start'])])
            kwargs_list.append(['end', self.format_date(self.kwargs['freq'], self.kwargs['end'])])
        except KeyError:
            pass
        return kwargs_list
    
    def format_date(self, freq, date):
        """formats input dates to correct"""
        date = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
        freq_dict = {'A': '%Y', 'M': '%Y%m', 'W': '%Y%m%d',
                     'D': '%Y%m%d', 'H': '%Y%m%dT%HZ'}
        formatted_date = datetime.strftime(date, freq_dict[freq])
        return formatted_date

In [53]:
#variables to specify date range and frequency
api_key = em.eia_api
start = '2016-01-01 01:00:00'
end = '2016-01-31 23:00:00'
freq = 'H'
# create list of series used for dictionary keys
keys = ['EBA.BPAT-ALL.D.H', 'EBA.AVA-ALL.D.H', 'EBA.NEVP-ALL.D.H', 
        'EBA.PGE-ALL.D.H ', 'EBA.PSEI-ALL.D.H', 'EBA.SCL-ALL.D.H ']
# create dict of GetSeriesRange objects from series list
series_dict = {key: GetSeriesRange(api_key=api_key, series_id=key, 
                                   start=start, end=end, freq=freq) for key in keys}

In [54]:
# loop over dictionary and print out full name of each series
for key in series_dict:
    print('{}: {}'.format(series_dict[key].data.series[0]['name'],key))

Demand for Puget Sound Energy, Inc. (PSEI), Hourly: EBA.PSEI-ALL.D.H
Demand for Portland General Electric Company (PGE), Hourly: EBA.PGE-ALL.D.H 
Demand for Bonneville Power Administration (BPAT), Hourly: EBA.BPAT-ALL.D.H
Demand for Avista Corporation (AVA), Hourly: EBA.AVA-ALL.D.H
Demand for Nevada Power Company (NEVP), Hourly: EBA.NEVP-ALL.D.H
Demand for Seattle City Light (SCL), Hourly: EBA.SCL-ALL.D.H 


In [55]:
# all the dataframes have the same date range so we can concatenate in a loop
concat_df = pd.concat([series_dict[key].data.df for key in series_dict], axis=1)
# and output as a pickle file - I'm doing this to avoid calling the EIA API as I develop
concat_df.to_pickle('data/nw_load_df.pkl')
# concat_df = pd.read_pickle('data/new_load_df.pkl')

In [56]:
concat_df.describe()

/home/johnr_000/venv_python3/lib/python3.5/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning:

Invalid value encountered in percentile



,EBA.PSEI-ALL.D.H,EBA.PGE-ALL.D.H,EBA.BPAT-ALL.D.H,EBA.AVA-ALL.D.H,EBA.NEVP-ALL.D.H,EBA.SCL-ALL.D.H
count,738.000000,743.00000,743.000000,743.000000,743.000000,743.000000
mean,3937.807588,2562.97712,7051.993271,1637.674293,3762.786003,1272.037685
std,554.123174,389.67134,871.093322,175.032909,276.749400,174.878074
min,2571.000000,1730.00000,5223.000000,1271.000000,3197.000000,869.000000
25%,NaN,2267.50000,6412.000000,1496.500000,3556.500000,1124.000000
50%,NaN,2615.00000,7083.000000,1677.000000,3726.000000,1306.000000
75%,NaN,2836.00000,7605.500000,1758.000000,3950.500000,1400.500000
max,5085.000000,3472.00000,9365.000000,2057.000000,4607.000000,1654.000000


In [57]:
cf.set_config_file(offline=False, world_readable=True, theme='pearl')
concat_df.iplot()

In [35]:
concat_df.head(50)

,EBA.PSEI-ALL.D.H,EBA.PACE-ALL.D.H,EBA.PACW-ALL.D.H,EBA.BPAT-ALL.D.H,EBA.PGE-ALL.D.H,EBA.SCL-ALL.D.H
2015-12-31 23:00:00,4207,5592,2819,7584,2856,1397
2015-12-31 22:00:00,4425,5626,2930,7702,2894,1425
2015-12-31 21:00:00,4628,5673,3035,7955,2946,1462
2015-12-31 20:00:00,4779,5735,3169,8276,3045,1491
2015-12-31 19:00:00,4947,5868,3323,8656,3102,1531
2015-12-31 18:00:00,5037,5870,3414,8991,3143,1573
2015-12-31 17:00:00,5047,5946,3457,9075,3122,1575
2015-12-31 16:00:00,4955,6187,3376,8945,3050,1533
2015-12-31 15:00:00,4689,5536,3186,8526,2855,1449
2015-12-31 14:00:00,4410,5566,2904,7959,2600,1301
